In [2]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url

#input
region:REGION='Metropolitana'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload())
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'{affected/clients:.2%}')

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=1) )
res

usuarios regionales: 2,467,769 clientes
usuarios sin servicio : 1,821  clientes 0.07%


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
71,Puente Alto,CGE,2024-08-16,751,1 days 15:53:07.794918
70,Puente Alto,CGE,2024-08-15,182,2 days 15:53:07.794918
92,Santiago,ENEL,2024-08-16,56,1 days 15:53:07.794918
36,Lampa,ENEL,2024-08-13,53,4 days 15:53:07.794918
82,Renca,ENEL,2024-08-16,49,1 days 15:53:07.794918
...,...,...,...,...,...
64,Pudahuel,ENEL,2024-08-07,1,10 days 15:53:07.795862
67,Pudahuel,ENEL,2024-08-14,1,3 days 15:53:07.795862
72,Puente Alto,EEPA,2024-08-16,1,1 days 15:53:07.795862
73,Quilicura,ENEL,2024-08-12,1,5 days 15:53:07.795862
